## 학습목표
1. selenium 모듈 사용법 알아보기

In [10]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

from bs4 import BeautifulSoup
import time

**selenium**
- 웹페이지 테스트 자동화용 모듈
- 개발/테스트용 드라이버(웹브라우저)를 사용하여 실제 사용자가 사용하는 것처럼 동작

**selenium 예제**
- python.org로 이동하여 자동으로 검색해보기
 1. python.org 사이트 오픈
 2. input 필드 검색하여 Key 이벤트 전달

In [22]:
chrome_driver = "C:\\Users\\LGE1\\chromedriver"
driver = webdriver.Chrome(chrome_driver)

driver.get('https://www.python.org')

search = driver.find_element_by_id('id-search-field')

search.clear()
time.sleep(3)

search.send_keys('lambda')

time.sleep(3)
search.send_keys(Keys.RETURN)
time.sleep(3)

driver.close()

**selenium을 이용한 다음뉴스 웹사이트 크롤링**
 - driver 객체의 find_xxx_by 함수 활용

In [27]:
chrome_driver = "C:\\Users\\LGE1\\chromedriver"
driver = webdriver.Chrome(chrome_driver)

url = 'https://news.v.daum.net/v/20190728165812603'
driver.get(url)

src = driver.page_source
soup = BeautifulSoup(src)

driver.close()

comment = soup.select_one('span.alex-count-area')
comment.get_text()

'42'

**selenium을 활용하여 특정 element의 로딩 대기**
- WebDriverWait 객체를 이용하여 해당 element가 로딩되는 것을 대기
- 실제로 해당 기능을 활용하여 거의 모든 사이트의 크롤링이 가능
- WebDriverWait(driver, 시간(초)).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'CSS_RULE')))

In [28]:
chrome_driver = "C:\\Users\\LGE1\\chromedriver"
driver = webdriver.Chrome(chrome_driver)

url = 'https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=105&oid=081&aid=0003018031'
driver.get(url)

WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'span.u_cbox_count')))

src = driver.page_source
soup = BeautifulSoup(src)

driver.close()

comment = soup.select_one('span.u_cbox_count')
comment.get_text()

'21'

## 학습목표
- 1. 다음 뉴스와 그 뉴스의 댓글 크롤링하기

In [31]:
import requests
from bs4 import BeautifulSoup

**뉴스 제목 크롤링**

In [34]:
def get_daum_news_title(news_id):
    url = 'https://news.v.daum.net/v/{}'.format(news_id)
    resp = requests.get(url)
    
    soup = BeautifulSoup(resp.text)
    title_tag = soup.select_one('h3.tit_view')
    
    if title_tag :
        return title_tag.get_text()
    return ''

In [35]:
get_daum_news_title('20190728165812603')

'일론머스크 "테슬라에서 넷플릭스·유튜브 즐길 날 온다"'

In [36]:
get_daum_news_title('kx9pacOTnC')

"'타격천재 격돌' 이정후 vs 강백호, 선수들은 누구의 손을 들어줬을까"

**뉴스 본문 크롤링**

In [37]:
def get_daum_news_content(news_id):
    url = 'https://news.v.daum.net/v/{}'.format(news_id)
    resp = requests.get(url)
    
    soup = BeautifulSoup(resp.text)

    content = ''
    for p in soup.select('div#harmonyContainer p'):
        content += p.get_text()

    return content

In [40]:
get_daum_news_content('20190728165812603')

'[아시아경제 이민우 기자] 일론 머스크 테슬라 최고경영자(CEO)가 자사 전기 자동차 모델에 넷플릭스와 유튜브 등 온라인동영상서비스(OTT)를 탑재할 것이라고 예고했다. 단순히 자율 주행 전기차가 단순히 주행을 위한 정보를 알려주는 것을 넘어 각종 영상 콘텐츠를 즐기는 공간으로도 확장하겠다는 전략으로 풀이된다.27일(현지시간) 더버지 등 주요 외신들에 따르면 머스크 CEO는 자신의 트위터를 통해 이 같은 계획을 밝혔다. 그는 "자동차가 정차했을 때 넷플릭스와 유튜브를 감상할 수 있는 기능이 조만간 추가될 것"이라며 "편안한 좌석과 서라운드 사운드 오디오를 통해 영화관과 같은 느낌을 받을 수 있을 것"이라고 강조했다.테슬라가 이처럼 콘텐츠 방면으로 확장하려 든 것은 이번이 처음이 아니다. 지난달 세계 최대 게임쇼 E3에서는 이미 운전자가 \'폴아웃 쉘터\'라는 게임을 할 수 있을 것이라고 발표한 바 있다. 이후에도 최근 게임업체 아타리사(社)의 자동차 경주 게임 ‘폴포지션’, 슈팅게임 ‘템페스트’, ‘미사일커맨드’ 등 고전 게임을 제공하기도 했다. 운전대로 게임을 조작하는 방식으로, 차가 주차돼 있을 경우에만 즐길 수 있다.이번 영상 콘텐츠는 주행 중에도 감상할 수 있도록 하는 방안을 고려하고 있다. 테슬라 측은 규제당국이 자율주행에 대해 완전히 승인하면 차량이 움직일 때에도 승객이 동영상을 즐길 수 있을 것이라고 설명했다.하지만 아직까지 자율주행차의 안전에 대한 우려는 완전히 걷혀지지 않은 상황이다. 지난 2017년 차량공유 서비스 우버의 자율주행 시범차량이 보행자와 충돌한 사고가 발생한 바 있다. 게다가 당시 시험 운전자는 디즈니의 동영상 스트리밍 서비스인 \'훌루\'를 이용하고 있던 것으로 밝혀졌다.이민우 기자 letzwin@asiae.co.kr<ⓒ경제를 보는 눈, 세계를 보는 창 아시아경제 무단전재 배포금지>'

In [39]:
get_daum_news_content('kx9pacOTnC')

'동아스포츠대상은 여러 시상식 중에서도 선수들이 직접 수상자를 뽑아 그 의미가 더 크다. 선별된 50명의 투표인단은 올 시즌 최고의 활약을 펼친 선수들에게 직접 투표했다. 공교롭게도 ‘타격천재’들의 싸움이 벌어졌다.먼저 올 시즌 타격왕을 차지한 키움 히어로즈 이정후(23)가 유력 후보로 부상했다. 이정후는 123경기에 출전해 타율 0.360, 7홈런, 84타점, 78득점의 성적을 거뒀다. 중견수로서 안정적 수비력도 뽐냈다. 그야말로 공수에 걸친 만점활약으로 팀의 가을야구 진출을 이끌었다.이정후는 11월 29일 열린 ‘2021 신한은행 SOL KBO 시상식’에서 타격왕을 수상하기도 했다. 이외에도 각종 연말 시상식을 휩쓸며 트로피 수집에 나선 상태다. 선수들이 직접 뽑는 대망의 동아스포츠대상에서도 올해의 선수상을 거머쥘지 관심이 모아진다.KT 위즈 강백호(22)는 이정후에 맞서는 가장 막강한 경쟁 후보다. 올 시즌 142경기에서 타율 0.347, 16홈런, 102타점, 76득점을 올리며 최고의 한 해를 보냈다. 전반기 95경기에선 타율 0.395를 기록하며 4할 타율에 도전하기도 했다.강백호는 타격왕 수상이 유력했으나 이정후의 후반기 맹추격을 뿌리치지 못한 채 결국 타이틀을 내줬다. 그러나 이정후보다 더 큰 성과를 만들어내며 2021시즌의 진정한 승자로 남았다. 팀의 정규시즌-한국시리즈 통합우승에 앞장섰다.타격천재들의 치열한 경쟁으로 끝날지, 아니면 베테랑 포수가 깜짝 반전에 성공할지로 관심을 모으는 동아스포츠대상 프로야구 올해의 선수상 수상자는 6일 공개된다. 장은상 기자 award@donga.com 저작권자(c)스포츠동아. 무단전재-재배포금지.'

**뉴스 댓글 크롤링**

In [54]:
url = 'https://comment.daum.net/apis/v1/posts/133493400/comments?parentId=0&offset=13&limit=10&sort=POPULAR&isInitial=false&hasNext=true&randomSeed=1638485338'
resp = requests.get(url)

resp.json()

[{'id': 393690060,
  'userId': -118726600,
  'postId': 133493400,
  'forumId': -99,
  'parentId': 0,
  'type': 'COMMENT',
  'status': 'S',
  'flags': 256,
  'rating': 0,
  'content': '시대는 이재 바야흐로  영적진화  와  과학 진화가 공존 하는 새상이   곳 도래 하갯군',
  'createdAt': '2019-07-28T18:48:57+0900',
  'updatedAt': '2019-07-28T18:48:57+0900',
  'childCount': 0,
  'likeCount': 0,
  'dislikeCount': 0,
  'recommendCount': 0,
  'screenedByKeeper': False,
  'user': {'id': -118726600,
   'status': 'S',
   'type': 'USER',
   'flags': 0,
   'icon': 'https://t1.daumcdn.net/profile/wIW3Q.DCC_g0',
   'url': '',
   'username': 'DAUM:82ada',
   'roles': 'ROLE_USER,ROLE_DAUM,ROLE_IDENTIFIED',
   'providerId': 'DAUM',
   'providerUserId': '82ada',
   'displayName': '생명의빚으로',
   'description': '',
   'commentCount': 7809}},
 {'id': 393751169,
  'userId': 25442700,
  'postId': 133493400,
  'forumId': -99,
  'parentId': 0,
  'type': 'COMMENT',
  'status': 'S',
  'flags': 4352,
  'rating': 0,
  'content': '운전할 땐 운전만 합시다',
  'c

In [58]:
def get_daum_news_comments(news_id):
    url_template = 'https://comment.daum.net/apis/v1/posts/{0}/comments?parentId=0&offset={1}&limit=10&sort=POPULAR&isInitial=false&hasNext=true&randomSeed=1638485338'
    offset = 0
    comments = []

    while True:
        url = url_template.format(news_id, offset)
        resp = requests.get(url)
        data = resp.json()
        if not data:
            break
        comments.extend(data)
        offset += 10
        
    return comments

In [62]:
len(get_daum_news_comments('133493400'))

15